<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline

import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Nonlinear Systems of Conservation Laws

We now turn to problems that are nonlinear and a system of equations.  This theory will combine ideas from both linear systems and nonlinear scalar equations.  In general this theory will split a single discontinuity into $m$ waves where $m$ is the number of equations.

This theory will presented primarily in the context of the 1-dimensional shallow water equations so that we can look at a concrete example.

## Shallow Water

The shallow water equations can most directly be derived from the invicid Navier-Stokes equations defined by
$$\begin{aligned}
    &u_x + w_z = 0 & & \text{incompressibility} \\
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_x & & \text{x-momentum} \\
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_z - g & & \text{z-momentum}
\end{aligned}$$
along with the boundary conditions
$$
    \left \{ \begin{aligned}
        &w = \eta_t + u \eta_x \\
        &P = P_a
    \end{aligned} \right . \quad \quad z = \eta
$$
and
$$
    w = ub_x \quad z = b
$$

### Non-Dimensionalization

Since the shallow water equations are based on a scaling with respect to "shallowness" we will descibe that exansion but reintroduce a dimsional version in closing.

To non-dimensionalize these equations we use the following scalings:
$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P}
$$
Using these scalings we can then define
$$
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
where $\epsilon$ the shallow water parameter relating wave-length to depth and $g$ is gravitational acceleration.

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Applying the scalings above we can transform the incompressibility equation to
$$
\begin{aligned}
    &u_x + w_z = 0 \Rightarrow \\
    &\frac{U}{\lambda} \widetilde{u}_{\widetilde{x}} + \frac{W}{a} \widetilde{w}_{\widetilde{z}} = 0 \Rightarrow \\
    &\widetilde{u}_{\widetilde{x}} + \frac{\epsilon U \lambda}{a U} \widetilde{w}_{\widetilde{z}} = 0 \Rightarrow \\
    &\widetilde{u}_{\widetilde{x}} + \widetilde{w}_{\widetilde{z}} = 0.
\end{aligned}
$$

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Applying the scalings again we can transform the x-momentum equation to
$$\begin{aligned}
    &u_t + (u^2)_x + (uw)_z = -\frac{1}{\rho} P_x \Rightarrow \\
    &\frac{U}{T} \widetilde{u}_{\widetilde{t}} + \frac{U^2}{\lambda} (\widetilde{u}^2)_{\widetilde{x}} + \frac{UW}{a} (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{P_0}{\rho \lambda} \widetilde{P}_{\widetilde{x}} \Rightarrow \\
    &\tilde{u}_{\widetilde{t}} + \frac{UT}{\lambda} (\widetilde{u}^2)_{\widetilde{x}} + \frac{TW}{a} (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{TP_0}{\rho \lambda U} \widetilde{P}_{\widetilde{x}} \Rightarrow \\
    &\tilde{u}_{\tilde{t}} + (\tilde{u}^2)_{\tilde{x}} + (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\frac{P_0}{\rho U^2} \widetilde{P}_{\widetilde{x}} 
\end{aligned}$$
If we now use the scaling $\frac{P_0}{\rho U^2} = 1$ then we arrive at the appropriate non-dimensional x-momentum PDE:
$$
    \widetilde{u}_{\widetilde{t}} + (\widetilde{u}^2)_{\widetilde{x}} + (\widetilde{u}\widetilde{w})_{\widetilde{z}} = -\widetilde{P}_{\widetilde{x}} 
$$

$$
    \frac{x}{\lambda} = \widetilde{x} \quad \quad \frac{z}{a} = \widetilde{z} \quad \quad \frac{t}{T} = \widetilde{t}\\
    \frac{u}{U} = \widetilde{u} \quad \quad \frac{w}{W} = \widetilde{w} \quad \quad \frac{P}{P_0} = \widetilde{P} \\
    \epsilon = \frac{a}{\lambda} \quad \quad U = \sqrt{ga} \quad \quad W = \epsilon U \quad \quad T = \frac{\lambda}{U}
$$
Finally applying the same procedure to the z-momentum equations we have
$$\begin{aligned}
    &w_t + (uw)_x + (w^2)_z = -\frac{1}{\rho} P_z - g \Rightarrow \\
    &\frac{W}{T}\widetilde{w}_{\widetilde{t}} + \frac{UW}{\lambda}(\widetilde{u}\widetilde{w})_{\widetilde{x}} + \frac{W^2}{a}(\widetilde{w}^2)_{\widetilde{z}} = -\frac{P_0}{\rho a} \widetilde{P}_{\widetilde{z}} - g \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + \frac{UT}{\lambda}(\widetilde{u}\widetilde{w})_{\widetilde{x}} + \frac{WT}{a}(\widetilde{w}^2)_{\widetilde{z}} = -\frac{T}{W} \left (\frac{P_0}{\rho a} \widetilde{P}_{\tilde{z}} + g \right ) \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} = -\frac{\lambda}{\epsilon U^2} \left (\frac{P_0}{\rho a} \widetilde{P}_{\widetilde{z}} + g \right ) \Rightarrow \\
    &\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} = -\frac{1}{g \epsilon^2} \left (g \widetilde{P}_{\widetilde{z}} + g \right ) \Rightarrow \\
    &\epsilon^2 \left (\widetilde{w}_{\widetilde{t}} + (\widetilde{u}\widetilde{w})_{\widetilde{x}} + (\widetilde{w}^2)_{\widetilde{z}} \right )= - \widetilde{P}_{\widetilde{z}} - g
\end{aligned}
$$

For the boundary conditions we also need to introduce these scalings along with $\frac{\eta}{\delta} = \widetilde{\eta}$ and $\frac{b}{\beta} = \widetilde{b}$:
$$\begin{aligned}
    &w = \eta_t + u \eta_x \Rightarrow \\
    &\epsilon U \widetilde{w} = \frac{\delta U}{\lambda} \widetilde{\eta}_{\widetilde{t}} + \frac{U\delta}{\lambda} \widetilde{u} \widetilde{\eta}_{\widetilde{x}} \Rightarrow \\
    &\epsilon \frac{\lambda}{\delta} \widetilde{w} = \widetilde{\eta}_{\widetilde{t}} + \widetilde{u} \widetilde{\eta}_{\widetilde{x}}
\end{aligned}$$
and
$$\begin{aligned}
    &w = u b_x \Rightarrow \\
    &\epsilon \frac{\lambda}{\beta} \widetilde{w} = \widetilde{u} \widetilde{b}_{\widetilde{x}}
\end{aligned}$$

For the upper boundary condition, if $\frac{\delta}{\lambda} = \mathcal{O}(\epsilon)$ the boundary condition is all of the same order.  This is probably not the case as $\delta \ll a$ unless in very shallow water.

The lower boundary condition is a similar situation, if $\frac{\beta}{\lambda} = \mathcal{O}(\epsilon)$ the boundary condition is all of the same order.  In the case of the bathymetry scale, this is probably the case as the bathymetry will vary on the scale of the depth scale $a$.

The final non-dimensionalized equations are then (dropping the $\widetilde{\cdot}$):
$$\begin{aligned}
    &u_x + w_z = 0 \\
    &u_t + (u^2)_x + (uw)_z = -P_x \\
    &\epsilon^2 \left (w_t + (uw)_x + (w^2)_z \right ) = -P_z - 1
\end{aligned}$$
with boundary conditions:
$$\begin{aligned}
        &\epsilon \frac{\lambda}{\delta}w = \eta_t + u \eta_x \quad P = P'_a \quad &z = \eta\\
        &\epsilon \frac{\lambda}{\beta}w = u b_x &z = b
\end{aligned}$$
where $P'_a$ is the appropriately scaled atmospheric pressure condition.

### Depth Integration

Integrating the non-dimensionalized incompressibility equations gives us:
$$\begin{aligned}
        &\int^{\eta}_b \left ( u_x + w_z \right) dz = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz - u \eta_x |_{z=\eta} + u b_x |_{z=b} + w|_{z=\eta} - w|_{z=b} = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz + \left [w - u \eta_x\right ]_{z=\eta} + \left [u b_x - w \right]_{z=b} = 0 \Rightarrow \\
        &\frac{\partial}{\partial x} \int^{\eta}_b u dz + \eta_t = 0 \Rightarrow \\
        &h_t + \frac{\partial}{\partial x} \int^{\eta}_b u dz = 0
\end{aligned}$$

Before we integrate the momentum equations we will introduce one of the two assumptions made to simplify the original equations.
1. The **hydrostatic assumption** assumes that the pressure is dominated by the gravitational force:
$$
    P = P_a + (\eta - z).
$$
1. The **differential advection** assumption assumes that there is little variation of the horizontal velocity $u$ through the water vertically.  This then implies that
$$
    \overline{f \cdot g} = \overline{f} \cdot \overline{g}
$$
where
$$
    \overline{(f \cdot g)} = \frac{1}{h} \int^\eta_b f \cdot g dz.
$$

Moving back to the integration of the momentum equations we will first start with the z-momentum equations, since the z-momentum equation is multiplied by what we will assume is a small parameter, i.e. $\epsilon \ll 1$, we can simplify the vertical momentum equation to
$$\begin{aligned}
    0 &= - P_z - 1 \\
    P_z &= -1 \\
    P &= P_a + (\eta - z),
\end{aligned}$$
which now justifies our assumption of hydrostatic equilibrium.

For the x-momentum equation we have now
$$\begin{aligned}
    &\int^{\eta}_b \left ( u_t + (u^2)_x + (uw)_z \right) dz = - \int^{\eta}_b (P_a + (\eta - z))_x dz \Rightarrow \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz - \eta_t u|_{z=\eta} + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz - \eta_x u^2|_{z=\eta} + b_x u^2|_{z=b} + uw|_{z=\eta} - uw|_{z=b} = - \int^{\eta}_b (\eta)_x dz \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz  - \left [u (\eta_t + \eta_x u - w) \right]_{z=\eta} + \left[u (b_x u - w)\right]_{z=b} = - \frac{\partial}{\partial x} \int^{\eta}_b \eta dz + \eta_x \eta - b_x \eta \\
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ -\eta h + \frac{1}{2} \eta^2 \right ] - b_x \eta \\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ -h^2 + hb  + \frac{1}{2} (h^2 + 2hb + b^2) \right ] - b_x (h+b) \Rightarrow \\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \int^{\eta}_b u^2 dz = \frac{\partial}{\partial x} \left[ - \frac{1}{2} h^2 \right ] - b_x h\\
        &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \left ( \int^{\eta}_b u^2 dz + \frac{1}{2} h^2 \right ) = - b_x h
\end{aligned}$$

Now if we use our differential advection assumption and simplify the notation
$$
    \overline{u} = \frac{1}{h} \int^{\eta}_b u dz \quad \quad
    \overline{u^2} = \frac{1}{h} \int^{\eta}_b u^2 dz
$$
we then have from the incompressibility integration
$$\begin{aligned}
    &h_t + \frac{\partial}{\partial x} \int^{\eta}_b u dz = 0 \\
    &h_t + (h \overline{u})_x = 0
\end{aligned}$$
and the x-momentum equation
$$\begin{aligned}
    &\frac{\partial}{\partial t} \int^{\eta}_b u dz + \frac{\partial}{\partial x} \left ( \int^{\eta}_b u^2 dz + \frac{1}{2} h^2 \right ) = - b_x h \\
    & (h\overline{u})_t + \left ( h \overline{u}^2 + \frac{1}{2} h^2 \right )_x = - b_x h.
\end{aligned}$$
Finally dropping the averaging bars we are lead to the shallow water equations:
$$\begin{aligned}
    &h_t + (hu)_x = 0 \\
    &(hu)_t + \left (hu^2 + \frac{1}{2} h^2 \right)_x = -h b_x
\end{aligned}$$

Although these above equations are a valid we will simplify our discussion and drop the term due to bathymetry $b$ and also reapply the dimensional version leading to our final form of the equations
$$\begin{aligned}
    &h_t + (hu)_x = 0 \\
    &(hu)_t + \left (hu^2 + \frac{1}{2} g h^2 \right)_x = 0
\end{aligned}$$
leading to a conservation law.

Note that we can also derive this set of equations from our more direct methods of considering fluxes but some assumptions must be made that makes these equations more difficult to justify.  See the textbook if you would like to this derivation.

Analyzing this equation in the context of systems of hyperbolic eqautions the first order of business is to compute the Jacobian of the flux function.  First define the vector of the solution as
$$
    q(x,t) = \begin{bmatrix} h \\ hu \end{bmatrix}
$$
and the flux function as
$$
    f(q) = \begin{bmatrix} hu \\ hu^2 + \frac{1}{2} gh^2 \end{bmatrix} = \begin{bmatrix} q^2 \\ \frac{(q^2)^2)}{q^1} + \frac{1}{2} g (q^1)^2 \end{bmatrix}.
$$
Note at this point that the flux is in-fact convex.

The quasi-linear form of the equations is defined by
$$
    q_t + f'(q) q_x = 0
$$
where
$$
    f'(q) = \begin{bmatrix}
        0 & 1 \\
        -(q^2 / q^1)^2 + g q^1 & 2 q^2 / q^1
    \end{bmatrix} = \begin{bmatrix}
        0 & 1 \\
        -u^2 + gh & 2u
    \end{bmatrix}.
$$
The eigensystem therfore has the structure
$$
    \lambda = u \pm \sqrt{gh} \text{ and } R = \begin{bmatrix}
        1 & 1 \\ u - \sqrt{gh} & u + \sqrt{gh} \end{bmatrix}
$$
These waves (speeds) are known as **gravity waves** (speeds) are are important for determining the behavior even in the simple case of a Riemann problem.  The non-dimensional parameter that is often used in these cases is often called the **Froude number** defined as}
$$
    \text{Fr} = \frac{|u|}{\sqrt{gh}}
$$
and is similar to the role of the speed of sound in gasses.

## Dam-Break and Riemann Problems

The **dam-break problem** is a special case of a Riemann problem where
$$
    h(x, 0) = \left \{\begin{aligned}
        h_\ell & & \text{if } x < 0 \\
        h_r & & \text{if } x > 0
    \end{aligned} \right .
$$
with $h_ell > h_r \geq 0$ and $u(x,0) = 0$.

In [ ]:
def dam_break_animation(h, u=[0.0, 0.0], N=100, ylimits=((0.0, 3.5), (-0.5, 2))):
   
    solver = pyclaw.ClawSolver1D(riemann.shallow_1D_py.shallow_fwave_1d)
    solver.kernel_language = "Python"
    
    solver.num_waves = 2
    solver.num_eqn = 2
    solver.fwave = True
    solver.limiters = [pyclaw.limiters.tvd.MC,
                       pyclaw.limiters.tvd.MC]
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.aux_bc_lower[0] = pyclaw.BC.extrap
    solver.aux_bc_upper[0] = pyclaw.BC.extrap

    x = pyclaw.Dimension(-5.0, 5.0, N, name='x')
    domain = pyclaw.Domain(x)
    state = pyclaw.State(domain, 2, 1)
    xc = domain.grid.x.centers
    state.q[0,:] = h[0] * (xc < 0) * numpy.ones(xc.shape) + h[1] * (xc >= 0) * numpy.ones(xc.shape)
    state.q[1,:] = u[0] * (xc < 0) * numpy.ones(xc.shape) + u[1] * (xc >= 0) * numpy.ones(xc.shape)
    state.q[1,:] *= state.q[0, :]
    state.aux[0, :] = numpy.zeros(xc.shape)
    
    state.problem_data['grav'] = 1.0
    state.problem_data['dry_tolerance'] = 1e-3
    state.problem_data['sea_level'] = 0.0
    
    claw = pyclaw.Controller()
    claw.tfinal = 2.0
    claw.num_output_times = 10
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver

    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig, axes = plt.subplots(1, 2)
    fig.set_figwidth(fig.get_figwidth() * 2)
    axes[0].set_xlim((x[0], x[-1]))
    axes[0].set_ylim(ylimits[0])
    axes[0].set_title(r"$h$")
    axes[1].set_xlim((x[0], x[-1]))
    axes[1].set_ylim(ylimits[1])
    axes[1].set_title(r"$hu$")

    def init():
        axes[0].set_xlim((x[0], x[-1]))
        axes[0].set_ylim(ylimits[0])
        h_line, = axes[0].plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        axes[1].set_xlim((x[0], x[-1]))
        axes[1].set_ylim(ylimits[1])
        hu_line, = axes[1].plot(x[0], claw.frames[0].q[1, :][0], 'bo-')
        
        return (h_line, hu_line)
    
    h_line, hu_line = init()
    
    def fplot(n):
        h_line.set_data([x,], [claw.frames[n].q[0, :]])
        hu_line.set_data([x,], [claw.frames[n].q[1, :]])
        axes[0].set_title(r"$h$ at $t = %s$" % claw.frames[n].t)
        axes[1].set_title(r"$hu$ at $t = %s$" % claw.frames[n].t)
        return (h_line, hu_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)

In [ ]:
HTML(dam_break_animation(h=[3.0, 1.0]).to_jshtml())

Now let's attempt to sketch the characteristics in this case.

## Characteristic Structure

As mentioned earlier, a nonlinear set of equations will have a different fluid (or particle) speeds from the characteristic speeds.  We can see in this in shallow water easily by noting that the fluid speed is directly $u$ and the characteristic speeds are $u \pm \sqrt{g h}$.

We also now have two characteristic families, the 1-characteristics where
$$
    \frac{\text{d} X}{\text{d} t} = \lambda^1 = u - \sqrt{g h}
$$
and the 2-characteristics where
$$
    \frac{\text{d} X}{\text{d} t} = \lambda^2 = u + \sqrt{g h}.
$$
We can again sketch both.  Note that the wave associated with each field defines either a rarefaction (in the 1-characteristic field) and a shock (in the 2-characteristic field).  The characteristics however are modified by the wave in the other field as expected.

Generally a system of $m$ equations will produce $m$ characteristic families and $m$ waves in the Riemann solution.

If we order the characteristic speeds 
$$
    \lambda^1 \leq \lambda^2 \leq \ldots \leq \lambda^{p-1} \leq \lambda^p \leq  \lambda^{p+1} \leq \ldots \leq \lambda^m
$$
and the $p$th wave is a shock, then the characteristics of families $1$ through $p-1$ will cross the wave from left to right and waves $p+1$ to $m$ will cross from right to left.  Most physical systems exhibit this behavior and are often called **classical Lax shocks**.

For this occur we need:
 - The system must be strictly hyperbolic
 - The conditions described later of **genuine nonlinearity**

## Two-Shock Riemann Solution

We now will turn to example situations for the shallow water equations with specific wave structures.  The first is the all shock (two-shock) Riemann problem with
$$
    u(x, 0) = \left \{\begin{aligned}
        u_\ell & & \text{if } x < 0 \\
        -u_\ell & & \text{if } x > 0
    \end{aligned} \right .
$$
with $u_\ell > 0$ and $h(x,0) \equiv h_0$ and $h_0 > 0$.

In [ ]:
HTML(dam_break_animation(h=[1, 1], u=[1, -1], ylimits=((0, 2.6), (-1.1, 1.1))).to_jshtml())

Let's again attempt to sketch the characteristics.

## Weak Waves and the Linearized Problem

We can of course linearize the shallow water equations by expanding around a background state and assuming the solution is a perturbation to this background state (similar to how we derived the acoustics equations).

For shallow water it makes the most sense to assume a non-zero depth and taking
$$
    h_\ell = H + \epsilon \quad h_r = H - \epsilon
$$
for $\epsilon \ll H$ and $u = \epsilon$.  This will then lead to two "acoustic" waves moving at $\pm \sqrt{g H}$ and look the same as a linear system.  What happens though if we plug this into a nonlinear solver with these initial conditions?

In [ ]:
H = 1.0
epsilon = 0.01
HTML(dam_break_animation(h = [H+epsilon, H-epsilon], u=[0.0, 0.0], N=500, ylimits=((H-10*epsilon, H+10*epsilon), (-1.1, 1.1))).to_jshtml())

## Solving the General Riemann Problem

1. Determine whether each of the two waves are a shock or rarefaction (entropy condition).
2. Determine the intermediate state $q_m$ between the waves.
3. Determine the structure of the rarefaction wave if present.

## Shock Waves and Hugoniot Loci

When we have a system of nonlinear equations things are not quite as straightforward as they were for linear systems.  Our main goal here is to connect the left state $q_\ell$ and $q_r$ to a middle state $q_m$.

To simplify the discussion consider a right going shock in the second family of characteristics.  This implies that the state $q_m$ and $q_r$ must be connected via the Rankine-Hugoniot condition:
$$
    s (q_* - q) = f(q_*) - f(q)
$$
leading to
$$\begin{aligned}
    s (h_* - h) &= h_* u_* - hu \\
    s (h_* u_* - h u) &= h_* u_*^2 - hu^2 + \frac{1}{2} g (h^2_* - h^2)
\end{aligned}$$
for the shallow water equations.  Here we are going to consider $h_*$ and $u_*$ fixed and given and $h$ and $u$ the set of valid states that can connect to $h_*$ and $u_*$ via a shock.  These admissable states $h$ and $u$ create curves in the phase plane called **Hugoniot loci**.

The easiest approach to satisfying this condition is to find $u$ as a function of $h$ and solve for $u$.
$$\begin{aligned}
    s = \frac{h_* u_* - h u}{h_* - h} & & \text{Use first equation} \\
    u^2 - 2 u_* u + \left[ u^2_* - \frac{g}{2} \left(\frac{h_*}{h} - \frac{h}{h_*} \right ) (h_* - h) \right ] = 0 & & \text{Substitute u and simplify} \\
    u(h) = u_* \pm \sqrt{\frac{g}{2} \left(\frac{h_*}{h} - \frac{h}{h_*} \right ) (h_* - h)} & & \text{Solve quadratic}
\end{aligned}$$

$$
    u(h) = u_* \pm \sqrt{\frac{g}{2} \left(\frac{h_*}{h} - \frac{h}{h_*} \right ) (h_* - h)}
$$
Observations:
1. If $h = h_*$ then $u = u_*$.
1. if $h \neq h_*$ then there are two solutions for $u$ corresponding to the two families of shocks.
1. If $q \approx q_*$ then the result will be approximately equal to the linearized theory and the Hugoniot loci will approximate the eigenvectors found from the linearization.

This final point is important as it serves as a way to tell which solution corresponds to which family as the eigenvector of the appropriate field will be tanget to the Hugoniot loci at $q_*$.

We can see this more clearly by multiplying the function $u(h)$ by $h$ and reformulating $h = h_* + \alpha$ to get
$$
    hu(h) = h_*u_* + \alpha \left[ u_* \pm \sqrt{g h_* \left(1 + \frac{\alpha}{h_*} \right ) \left(1 + \frac{\alpha}{2h_*} \right ) }\right ].
$$
At $\alpha = 0$ we end up having
$$
    q = q_* + \alpha \begin{bmatrix} 1 \\ u_* \pm \sqrt{g h_* + \mathcal{O}(\alpha) }\end{bmatrix} \quad \text{as } \alpha \rightarrow 0.
$$

### All-Shock Riemann Solution Revisited

So we have something to connect to with our current knowledge let us suppose that both the first and second fields contain shocks.  We can then find $q_m$ by seeing which state $q_m$ can connect to both $q_\ell$ and $q_r$.

First for the right state we have
$$\begin{aligned}
    h_m u_m &= h_r u_r + (h_m - h_r) \left[ u_r + \sqrt{g h_r \left(1 + \frac{h_m - h_r}{h_r} \right) \left(1 + \frac{h_m - h_r}{2 h_r} \right ) } \right ] \\
    u_m &= u_r + (h_m - h_r) \sqrt{\frac{g}{2} \left( \frac{1}{h_m} + \frac{1}{h_r} \right) }.
\end{aligned}$$
Similarly we also can connect to the left state via
$$
    u_m = u_\ell - (h_m - h_\ell) \sqrt{\frac{g}{2} \left( \frac{1}{h_m} + \frac{1}{h_\ell} \right) }
$$
We now have two equations with two unknows.

The general approach at this point is to set both conditions equal to each other and derive a value for $h_m$ by some nonlinear solver and then subsequently finding $u_m$

### Entropy Conditions

What if we apply the above procedure to a problem that we know only contains 1 shock such as the dam-break problem.  At this point we do not have a means to distinguish between these cases as both the all-shock, all-rarefaction and mixed shock-rarefaction solutions are all weak solutions.  We need to turn to entropy conditions to distinguish then the relevant solution.

A discontinuity separating states $q_\ell$ and $q_r$, propagating at speed $s$, statisfies the **Lax entropy condition** if there is an index $p$ such that
$$
    \lambda^p(q_\ell) > s > \lambda^p(q_r)
$$
so that $p$-characteristics are impinging on the discontinuity, while the other characteristics are crossing the discontinuity,
$$\begin{aligned}
    \lambda^j(q_\ell) < s & &\text{and}& &\lambda^j(q_r) < s \quad \text{for } j < p \\
    \lambda^j(q_\ell) > s & &\text{and}& &\lambda^j(q_r) > s \quad \text{for } j > p.
\end{aligned}$$

This condition only works however for strictly hyperbolic PDEs where each field is genuinely nonlinear (more on that later).

As is common another, more general condition can be formulated for a particular system.  For shallow water a 1-shock connecting $q_\ell$ and $q_m$ must have the characteristic speed
$$
    \lambda^1 = u - \sqrt{g h}
$$
decrease as we expect the characteristics must impinge on the shock in the first characteristics family.  Adding in the Rankine-Hugoniot condition this implies also that $h$ must incrase implying that we require
$$
    h_m > h_\ell.
$$
Similarly the 2-shock connecting $q_m$ to $q_r$ must sastisfy
$$
    h_m > h_r.
$$

## Rarefactions

The second type of wave we need to investigate for Riemann problems are rarefactions.  One particular type of rarefactions are important for us in the Riemann problems.  **Centered rarefactions** are constant along every ray $x/t = $ constant as we expect.

In [ ]:
# All rarefaction solution
HTML(dam_break_animation(h=[1, 1], u=[-1.0, 1.0], ylimits=((0, 2.6), (-1.1, 1.1))).to_jshtml())

Like before we can solve for the rarefaction by assuming that the solution is a similarity solution:
$$
    q(x, t) = \widetilde{q}(\xi)
$$
where
$$
    \xi = \frac{x}{t}
$$
leading to
$$
    f'(\widetilde{q}(x / t)) \widetilde{q}'(x / t) = \xi \widetilde{q}'(\xi).
$$
For systems now this is generally a system of equations as $\widetilde{q}$ is now a vector meaning that solving the problem implies solving an eigenproblem.

### Integral Curves

Assume that $\widetilde{q}(\xi)$ is a smooth curve through state space parameterized by $\xi$.  This is called an **integral curve of the vector field** $r^p$ if at each point $\widetilde{q}(\xi)$ the tangent vector, $\widetilde{q}'(\xi)$ is an eigenvector of the Jacobian $f'(\widetilde{q}(\xi))$ corresponding to the eigenvalue $\lambda^p(\widetilde{q}(\xi))$.

If we pick out one set of eigenvectors $r^p(q)$ then
$$
    \widetilde{q}'(\xi) = \alpha(\xi) r^p(\widetilde{q}(\xi))
$$
where the value of $\alpha(\xi)$ depends on the parameterization of the curve used and the normalization of $r^p$.

For the shallow water equations let's examine the eigenvectors associated with $r^2$.  We know then that
$$
    \widetilde{q}'(\xi) = r^2(\widetilde{q}(\xi)) = \begin{bmatrix}
        1 \\ \frac{\widetilde{q}^2}{\widetilde{q}^1} + \sqrt{g \widetilde{q}^1}
    \end{bmatrix}
$$
where $\alpha \equiv 1$ in this case.  This leads to 2 ODEs
$$\begin{aligned}
    &(\widetilde{q}^1)' = 1 \\
    &(\widetilde{q}^2)' = \frac{\widetilde{q}^2}{\widetilde{q}^1} + \sqrt{g \widetilde{q}^1}.
\end{aligned}$$
Try now to solve these equations.

The first equation implies
$$
    \widetilde{q}^1(\xi) = \xi.
$$
Plugging this into the second equation leads to
$$
    (\widetilde{q}^2)' = \frac{\widetilde{q}^2}{\xi} + \sqrt{g \xi}.
$$
If we now fix one point $(h_*, u_*)$ on the integral curve requiring
$$
    \widetilde{q}^2(h_*) = h_* u_*
$$
which leads to the solution
$$
    \widetilde{q}^2(\xi) = \xi u_* - 2 \xi (\sqrt{g h_*} - \sqrt{g \xi}).
$$
Since $\xi \equiv h$ is the parameterization we can also write this as
$$
    hu = hu_* - 2 h (\sqrt{g h_*} - \sqrt{g h})
$$
or simplifying a bit
$$
    u = u_* - 2 (\sqrt{g h_*} - \sqrt{g h}).
$$
Similarly for the $r^1(\xi)$ field the final equation is
$$
    u = u_* + 2 (\sqrt{g h_*} - \sqrt{g h}).
$$

### Riemann Invariants

We now have expressions for the integral curves of both $r^1$ and $r^2$ where we have an arbitrary point on the curve $(h_*, u_*)$.  We can of course rewrite this condition as
$$
    u + 2\sqrt{g h} = u_* + 2 \sqrt{g h_*}
$$
implying that there for any two points on the curve the value above must be constant, in other words invariant on the curve.

We can extend this to a more formal definition so that for any point on the integral curve, there is a **Riemann invariant** function for each family, which remains constant on that curve.  For the 1-family we have
$$
    w^1(q) = u + 2 \sqrt{g h}
$$
and
$$
    w^2(q) = u - 2 \sqrt{g h}
$$
for the 2-family.  These are also sometimes called $p$**-Riemann invariants**.

Checking back with our original supposition that we have integral curves we can make the following statements.  If $\widetilde{q}(\xi)$ is our parameterization of any $r^p$ integral curve, then $w^p(q)$ is constant along $\widetilde{q}(\xi)$ implying
$$
    \frac{\text{d}}{\text{d} \xi} w^p(\widetilde{q}(\xi)) = 0.
$$
Extending this out to all fields leads to the broader condition that
$$
    \nabla w^p(\widetilde{q}(\xi)) \cdot \widetilde{q}'(\xi) = 0
$$
where $\nabla w^p$ is the gradient with respect to $q$ and therefore
$$
    \nabla w^p(\widetilde{q}(\xi)) \cdot r^p(\widetilde{q}(\xi)) = 0,
$$
which must hold on every integral curve.  In other words the Riemann invariants must be orthogonal to the eigenvectors $r^p$ at each point $q$ or that the integral curves are level sets of the functions $w^p(q)$.

### Simple Waves

A solution to a conservation law for which
$$
    q(x,t) = \widetilde{q}(\xi(x, t))
$$
where $\widetilde{q}(\xi)$ traces out integral curves of some family and $\xi(x,t)$ is some smooth mapping $(x,t) \mapsto \xi$ is also called a **simple wave**.

If we want to find the function $\xi(x,t)$ that satisfies the conservation law (rather than just assume it as we have done in the past) we can derive the appropriate mapping by plugging in the function $\xi(x,t)$ into the conservation law.  Do this now to derive what PDE the function $\xi(x,t)$ must satisfy.

$$\begin{aligned}
    q_t = \widetilde{q}'(\xi) \xi_t \text{ and } q_x = \widetilde{q}'(\xi) \xi_x & & \text{Take derivatives of } q \\
    \widetilde{q}'(\xi) \xi_t + f'(\widetilde{q}(\xi)) \widetilde{q}'(\xi) \xi_x = 0 & & \text{Plug into conservation law} \\
    \left[ \xi_t + f'(\widetilde{q}(\xi)) \xi_x \right] \widetilde{q}'(\xi) = 0 & & \text{Factor out eigenvector} \\
    \left[ \xi_t + \lambda^p(\widetilde{q}(\xi)) \xi_x \right] \widetilde{q}'(\xi) = 0 & & \text{Eigenproblem} \\
    \xi_t + \lambda^p(\widetilde{q}(\xi)) \xi_x  = 0 & & \text{} \\
\end{aligned}$$

Suppose we choose initial data $q(x,0)$ that lies on the integral curve where
$$
    q(x,0) = \widetilde{q}_0(\xi(x))
$$
implying that we also have $\xi(0)$ so that we have initial data for our new PDE.  If $\lambda^p(\widetilde{q}(\xi(x,0)))$ is monotonically increasing then the characteristics will always be spreading out and we do not expect to see a breakdown of the solution.

### Genuine Nonlinearity and Linear Degeneracy

One condition that has been mentioned previously is the idea that we want $\lambda^p(\widetilde{q}(\xi))$ to vary monotonically as we move along an integral curve.  This condition is similar to the condition that the flux function is convex and for systems this condition generally replaces the convexity condition as an analogous condition.  We therefore say that the $p$th field is **genuinely nonlinear** if $\lambda^p(\widetilde{q}(\xi))$ varies monotonically with $\xi$ along every integral curve.

Note this variation can be computed by
$$
    \frac{\text{d}}{\text{d} \xi} \lambda^p(\widetilde{q}(\xi)) = \nabla \lambda^p(\widetilde{q}(\xi)) \cdot \widetilde{q}'(\xi).
$$

Since $\widetilde{q}'(\xi) = \alpha r^p(\widetilde{q}(\xi))$ we know that the $p$the field is genuinely nonlinear if
$$
    \nabla \lambda^p(q) \cdot r^p(q) \neq 0 \quad \forall q.
$$

Also if
$$
    \nabla \lambda^p(q) \cdot r^p(q) \equiv 0 \quad \forall q.
$$
then $\lambda^p(q)$ is constant along each curve, an example where this occurs is for a constant-coefficient linear hyperbolic system.  Regardless if this occurs we say that the field is **linearly degenerate** and acts like a linear wave.

### Centered Rarefaction Waves

A **centered rarefaction wave** is a rarefaction that is genuinely nonlinear and for which $\xi(x,t) = x/t$ such that the solutions is constant on rays through the origin and has the form
$$
    q(x,t) = \left \{ \begin{aligned}
        q_\ell & & \text{if } x/t \leq \xi_1 \\
        \widetilde{q}(x/t) & & \text{if } \xi_1 \geq x/t \leq \xi_2 \\
        q_r & & \text{if } x/t \geq \xi_2 \\
\end{aligned} \right .
$$
Note that this does imply that $\lambda^p(q_\ell) < \lambda^p(q_r)$ so that the characteristics spread out.

By definition we are also forced to pick the parameterization
$$
    \xi = x/t
$$
implying that the value $\widetilde{q}(\xi)$ along $x / t = \xi$ is propagating at speed $\xi$.  This also implies that the characteristic speed $\lambda^p(\widetilde{q}(\xi))$ and $\xi$ must agree.  We can also see this as our PDE for $\xi$ is for this case
$$\begin{aligned}
    -\frac{x}{t^2} + \lambda^p(\widetilde{q}(x/t)) \frac{1}{t} = 0 \Rightarrow \\ \frac{x}{t} = \lambda^p(\widetilde{q}(x/t)).
\end{aligned}$$

The more important point here is that the edges of the rarefaction fan should be the ray $x/t = \lambda^p(q_*)$ implying that we can also identify $\xi_1$ and $\xi_2$ as
$$
    \xi_1 = \lambda^p(q_\ell) \quad \xi_2 = \lambda^p(q_r).
$$

To figure out how the value of $\widetilde{q}(x/t)$ varies inside of the rarefaction we can take the following steps:
$$\begin{aligned}
    \xi = \lambda^p(\widetilde{q}(\xi)) & & \text{Implied from above} \\
    1 = \nabla \lambda^p(\widetilde{q}(\xi)) \cdot \widetilde{q}'(\xi) & & \text{Differentiate w.r.t. }\xi \\
    1 = \alpha(\xi) \nabla \lambda^p(\widetilde{q}(\xi)) \cdot r^p(\widetilde{q}'(\xi)) & & \text{Plug in } \widetilde{q}'(\xi) = \alpha r^p \\
    \alpha(\xi) = \frac{1}{\nabla \lambda^p(\widetilde{q}(\xi)) \cdot r^p(\widetilde{q}(\xi))} & & \text{Solve for } \alpha(\xi) \\
    \widetilde{q}'(\xi)) = \frac{r^p(\widetilde{q}(\xi))}{\nabla \lambda^p(\widetilde{q}(\xi)) \cdot r^p(\widetilde{q}(\xi))} & & \text{Plug in }\alpha \text{ in definition}
\end{aligned}$$
Note now that we see the importance of a monotonically varying, nonzero condition as expressed in the denominator here.

#### Example:  Shallow Water

For shallow water compute the solution for the expression through the rarefaction.  Note that you will need
$$
    \lambda^1 = u - \sqrt{gh} = \frac{q^2}{q^1} - \sqrt{g q^1} \quad r^1 = \begin{bmatrix} 1 \\ \lambda^1 \end{bmatrix}
$$
or
$$
    \lambda^2 = u + \sqrt{gh} = \frac{q^2}{q^1} + \sqrt{g q^1} \quad r^2 = \begin{bmatrix} 1 \\ \lambda^2 \end{bmatrix}.
$$
Depending on the direction you go.

Approaching this from the left we need to compute
$$
    \nabla \lambda^1 = \begin{bmatrix}
        -\frac{q^2}{(q^1)^2} - \frac{1}{2} \sqrt{\frac{g}{q^1}} \\
        \frac{1}{q^1}
    \end{bmatrix}.
$$
The inner product of the gradient and eigenvector is then
$$\begin{aligned}
    \nabla \lambda^1 \cdot r^1 &= \begin{bmatrix}
        -\frac{q^2}{(q^1)^2} - \frac{1}{2} \sqrt{\frac{g}{q^1}} \\
        \frac{1}{q^1}
    \end{bmatrix} \cdot \begin{bmatrix} 1 \\  \frac{q^2}{q^1} - \sqrt{g q^1} \end{bmatrix} \\ 
    &= -\frac{q^2}{(q^1)^2} - \frac{1}{2} \sqrt{\frac{g}{q^1}} + \frac{q^2}{(q^1)^2} - \sqrt{\frac{g}{q^1}} \\
    &= -\frac{3}{2} \sqrt{\frac{g}{q^1}}
\end{aligned}$$

Plugging this into our equation for $\widetilde{q}'$ we get
$$
    \widetilde{q}' = -\frac{2}{3} \sqrt{\frac{\widetilde{q}^1}{g}} \begin{bmatrix}
        1 \\ \frac{\widetilde{q}^2}{\widetilde{q}^1} - \sqrt{g \widetilde{q}^1}
    \end{bmatrix}
$$
The first equation implies
$$
    \widetilde{h}'(\xi) = -\frac{2}{3} \sqrt{\frac{\widetilde{h}(\xi)}{g}}
$$
whose general solution is
$$
    \widetilde{h} = \frac{1}{9g} (c - \xi)^2
$$
where $c$ is some constant determined by the initial condition $\xi_1$ or that $\widetilde{h} = h_\ell$ at $\xi = u_\ell - \sqrt{g h_\ell}$ and $\widetilde{h} = h_r$ at $\xi = u_r - \sqrt{g h_r}$.  This can be satisfied if
$$
    c = u_\ell + 2 \sqrt{g h_\ell} = u_r + 2 \sqrt{g h_r}
$$
which of course is our Riemann invariant.  We can then also observe that $\widetilde{h}$ varies quadratically.

Once we know $\widetilde{h}$ we can of course use our the Riemann invariant to also then find $\widetilde{u}$.

### The All-Rarefaction Solution

Extending beyond the single rarefaction now consider the case where both waves are rarefactions and we want to find the middle state $q_m$ as we did before with the all-shock problem.

Since we know that the integral curves connecting a state $q_*$ to $q$ must satisfy
$$
    u = u_* \pm 2 (\sqrt{g h_*} - \sqrt{g h})
$$
we can fill in the values here for the 1-field to find
$$
    u_m = u_\ell + 2 (\sqrt{g h_\ell} - \sqrt{g h_m})
$$
and for the 2-field
$$
    u_m = u_r - 2 (\sqrt{g h_r} - \sqrt{g h_m})
$$
Equating these two equations leads to
$$
    u_\ell + 2 (\sqrt{g h_\ell} - \sqrt{g h_m}) = u_r - 2 (\sqrt{g h_r} - \sqrt{g h_m})
$$
whose solution is
$$
    h_m = \frac{1}{16 g} \left[u_\ell - u_r + 2(\sqrt{g h_\ell} + \sqrt{g h_r}) \right]^2.
$$
With this in hand we can easily calculate $u_m$.

## Solving the General Dam-Break Problem

Finally let's consider the dam-break problem originally discussed.  Here we expect a rarefaction and shock in the 1 and 2 characteristic families respectively if $h_\ell > h_r$.  We will focus on this case for simplicity.

First let's start with the value across the 1-wave rarefaction.  We know that the state connecting $q_\ell$ and $q_m$ must lie on the integral curve and therefore is governed by
$$
    u_m = u_\ell + 2 (\sqrt{g h_\ell} - \sqrt{g h_m}).
$$

Similarly we know that $q_m$ must also lie on the same Hugoniot locus as $q_r$ such that
$$
    u_m = u_r + (h_m - h_r) \sqrt{\frac{g}{2} \left( \frac{1}{h_m} + \frac{1}{h_r} \right)}.
$$

As before we can combine these equations together to find one nonlinear equation for $h_m$
$$
    u_\ell + 2 (\sqrt{g h_\ell} - \sqrt{g h_m}) = u_r + (h_m - h_r) \sqrt{\frac{g}{2} \left( \frac{1}{h_m} + \frac{1}{h_r} \right)}.
$$

At this point we really cannot do anything in general but use a numerical solver.  Sometimes there are symmetries and simplifications that may make this equation easier to solve and often practical solvers will take advantage of this.

One final point that is worthwhile noting, often times the Hugoniot loci and and integral curves are very similar but not usually identical.  If they are identical this is a special type of system called a **Temple-class system** and is a subject of some study.

## The General Riemann Solver for Shallow Water

Now that we have considered most of the different situations we can find ourselves in for the shallow water equations let's now consider a more general approach to solving the problem.

One useful construct for solving the general Riemann problem is to define the following functions
$$
    \phi_\ell = \left \{ \begin{aligned}
        u_\ell + 2 (\sqrt{g h_\ell} - \sqrt{g h}) & & \text{if } h < h_\ell \\
        u_\ell - (h - h_\ell) \sqrt{\frac{g}{2} \left( \frac{1}{h} + \frac{1}{h_\ell} \right ) } & & \text{if } h > h_\ell
    \end{aligned} \right .
$$
and
$$
    \phi_r = \left \{ \begin{aligned}
        u_r - 2 (\sqrt{g h_r} - \sqrt{g h}) & & \text{if } h < h_r \\
        u_r + (h - h_r) \sqrt{\frac{g}{2} \left( \frac{1}{h} + \frac{1}{h_r} \right ) } & & \text{if } h > h_r
    \end{aligned} \right ..
$$
This allows us to simply look at the values of the depth and apply the correct function to find the middle state $q_m$ effectively building in an entropy condition.